In [1]:
import sentencepiece as sp

In [3]:
import os

''' Concatenate the contents of all files in the data folder into one single text file named full.txt. '''
def concat_files():
    with open('full.txt', 'w', encoding='utf8') as f:
        for file in os.listdir('data'):
            with open(os.path.join('data', file), encoding='utf8') as g:
                f.write(g.read())

concat_files()

In [ ]:
sp.SentencePieceTrainer.train(input='full.txt', model_prefix='m_en_de', vocab_size=15000, model_type='unigram', character_coverage=1.0, input_sentence_size=10000000, shuffle_input_sentence=True, max_sentence_length=1000, num_threads=16, unk_id=0, bos_id=1, eos_id=2, pad_id=3)

In [1]:
import torch
import numpy as np

c:\Users\Chen Zerui\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ckpt = torch.load('experiments/en-de-v1/model-epoch=02-val_loss=5.87.ckpt', map_location='cpu')

In [6]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.tril(np.ones(attn_shape)).astype('uint8')
    return torch.from_numpy(subsequent_mask)

In [7]:
subsequent_mask(10)

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], dtype=torch.uint8)

In [8]:
causal_mask = torch.tril(torch.ones(10, 10, dtype=bool))

In [10]:
causal_mask & False

tensor([[False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False]])